In [1]:
import tensorflow as tf
import pickle, pandas as pd, re, numpy as np, ast, warnings

from joblib import Parallel, delayed

import time
from copy import deepcopy
from collections import defaultdict, OrderedDict
from itertools import chain, starmap
from itertools import product
import unicodedata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

from TurkishStemmer import TurkishStemmer
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from textblob import TextBlob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()

In [3]:
df_full = pd.read_csv("../datasets/movie_data.csv")
df_full.head()

,Language,Movie_ID,Review,Score
0,en,-800777728,i love science fiction and i hate superheroes ...,9
1,en,-800777728,the movie is absolutely incredible all the per...,10
2,en,-1018312192,in a cinematic era dominated by reboots and mi...,8
3,en,-1018312192,movie review on rise of the planet of the apes...,4
4,en,-1018312192,during experiments to find a cure for alzheime...,7


In [4]:
df_full.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,29,29,29
2,21,21,21
3,14,14,14
4,23,23,23
5,83,83,83
6,43,43,43
7,71,71,71
8,207,207,207
9,175,175,175


In [5]:
# en_vects = gensim.models.KeyedVectors.load_word2vec_format(r"../../NLP_data/GoogleNews-vectors-negative300.bin", binary=True)
en_vects = gensim.models.KeyedVectors.load_word2vec_format(r"../../NLP_data/wiki.en/wiki.en.vec", binary=False)

In [6]:
tr_vects = gensim.models.KeyedVectors.load_word2vec_format(r"../../NLP_data/wiki.tr/wiki.tr.vec", binary=False)

In [7]:
turkish_stemmer = TurkishStemmer()
def clean(text, language="en", stem=True):
    global turkish_stemmer
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').lower().decode("ascii")
    
    if language == "tr":
        if stem:
            text= ' '.join([turkish_stemmer.stem(w) for w in text.split()])
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r'[0-9]', '#', text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"e(\s)?-(\s)?mail", "email", text)

    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    return TextBlob(text)


In [8]:
VECTOR_SIZE = 300
def vectorize(text, language):
    global VECTOR_SIZE            
    blob = clean(text, language)
    vector = np.zeros(VECTOR_SIZE)
    if len(blob.words) < 1:
        return None

    for word in blob.words:
        try:
            if language == "en":
                vector += globals()["en_vects"][word]
            else:
                vector += globals()["tr_vects"][word]
        except KeyError as e:
#             warnings.warn(str(e))
            continue
    vector /= max(len(blob.words),1)
    return vector

In [9]:
def getvec(x):
    lang, rev = x.split(":::::")
    return vectorize(rev, lang)

In [10]:
# LMSR
def preprocess_data(df, language_column="Language", review_column="Review"):
    LMSR_df = df.copy()
    LMSR_df["lang_rev"] = LMSR_df[[language_column, review_column]].apply(lambda x: x[0]+":::::"+x[1], axis=1)
    LMSR_df["rev_vec"] = LMSR_df["lang_rev"].apply(lambda x:getvec(x))
    LMSR_df.drop(["lang_rev", "Review"], axis=1, inplace=True)
    return LMSR_df

In [11]:
def distance_accuracy(y_true, y_predict):
    res = 0
    for i in range(len(y_true)):
        res += abs(y_true[i]-y_predict[i])
    return 1-res/(len(y_true)*len(set(y_true)))

In [12]:
def get_XYy(LMSR):
    X = np.zeros((len(LMSR), VECTOR_SIZE))
    Y = np.zeros((len(LMSR), VECTOR_SIZE))
    y = np.zeros((len(LMSR)))
    i = 0
    for rev in LMSR.iterrows():
        score = rev[1][2]
        rev_vec = rev[1][3]
        score_vec = rev[1][4]

        X[i] = rev_vec
        Y[i] = score_vec
        y[i] = score

        i += 1
    return X, Y, y

In [13]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [14]:
def sigmoid(x, derive=False):
    if derive:
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [15]:
def get_data_dict(df, get_L2and3=False):
    data_dict = dict() #{language:{score: {movie_id: [rev1, rev2, ..., revn]}}}
    L1 = dict()  # {(languge, score, movie_id): list of reviews with the same score with the same language}
    L2 = dict()  # {(language, score): None}
    L3 = dict()  # {score: None}
    for _, row in df.iterrows():
        lang = row["Language"]
        movie_id = row["Movie_ID"]
        score = row["Score"]
        review = row["rev_vec"]

        data_dict.setdefault(lang, {})
        data_dict[lang].setdefault(score, {})
        data_dict[lang][score].setdefault(movie_id, [])
        data_dict[lang][score][movie_id].append(review)
        
        L1.setdefault((lang, score, movie_id), list())
        L1[(lang, score, movie_id)].append(review)
        if get_L2and3:    
            L2[(lang, score)] = None
            L3[score] = None
    if get_L2and3:
        return data_dict, L1, L2, L3
    return data_dict, L1

In [16]:
def get_L2(LSM_R, data_dict):
    L2 = dict()  # {(language, score): list of movies vectors}
    for language in data_dict:
        for score in data_dict[language]:
            for movie_id in data_dict[language][score]:
                L2.setdefault((language, score), list())
                L2[(language, score)].append(LSM_R[(language, score, movie_id)])
    return L2

In [17]:
def get_L3(LS_MR, data_dict):
    L3 = dict()  # {score: vector of merged languages for that score}
    for language in data_dict:
        for score in data_dict[language]:
            L3.setdefault(score, list())
            L3[score].append(LS_MR[(language, score)])
    return L3

In [18]:
def merge(L, W):
    merged = dict()  # {item: vector of merged subitems}
    for i, item in enumerate(sorted(L)):
        for subitem in L[item]:
            merged.setdefault(item, [np.zeros(VECTOR_SIZE),0])
            merged[item][0] += sigmoid(subitem.dot(W[i]))
            merged[item][1] += 1
    for item in merged:
        merged[item] = merged[item][0]/ merged[item][1]
    return merged

In [19]:
def update_weights(L, delta, W, alpha=0.1):
    for i, k in enumerate(sorted(L)):
        for l in L[k]:
            W[i] += l.T.dot(delta[i]) *alpha
    return W

In [20]:
def get_layer_error(delta, W):
    error = 0
    for i in range(len(delta)):
        error += delta[i].dot(W[i].T)
    return error/len(delta)

In [21]:
def get_layer_delta(error, layer, size):
    delta = np.zeros((size, VECTOR_SIZE))
    j = 0
    for i,k in enumerate(sorted(layer)):
        for l in layer[k]:
            delta[j] = error[i]*sigmoid(l, True)
            j += 1
    return delta

## Training

## Full-batch

In [22]:
def get_score_vects(df, iterations=100, alpha=0.1, random_state=42, W1=None, W2=None, W3=None, W4=None):
    LSMR = preprocess_data(df)
    data_dict, L1 = get_data_dict(LSMR)
    y = softmax(list(LSMR.Score))
#     np.random.seed(random_state)
    learning_curve = dict()
    for i in range(iterations+1):
        # forward propagation
        if W1 is None:
            W1 = 2*np.random.random((len(L1), 300, 300))-1

        LSM_R = merge(L1, W1)
        L2 = get_L2(LSM_R, data_dict)
        if W2 is None:
            W2 = 2*np.random.random((len(L2), 300, 300))-1

        LS_MR = merge(L2, W2)
        L3 = get_L3(LS_MR, data_dict)
        if W3 is None:
            W3 = 2*np.random.random((len(L3), 300, 300))-1

        score_vectors_dict = merge(L3, W3)
        l4 = sigmoid(np.array([v for k, v in sorted(score_vectors_dict.items())]))
        if W4 is None:
            W4 = 2*np.random.random((300, len(LSMR)))-1
        
        l5 = softmax(l4.dot(W4))  # predicted scores
        
        # Calculate the error
        l5_error = np.mean(np.dot(np.log(l5), y))
        
        # Back propagation
        l5_delta = l5_error * sigmoid(l5, True)
        W4 += l4.T.dot(l5_delta)*alpha
        
        l4_error = l5_delta.dot(W4.T)
        l4_delta = l4_error * sigmoid(l4, True)
        
        W3 = update_weights(L3, l4_delta, W3, alpha)
        
        l3_error = get_layer_error(l4_delta, W3)
        l3_delta = get_layer_delta(l3_error, L3, len(L2))
        
        W2 = update_weights(L2, l3_delta, W2, alpha)
        
        l2_error = get_layer_error(l3_delta, W2)
        l2_delta = get_layer_delta(l2_error, L2, len(LSMR))
        
        W1 = update_weights(L1, l2_delta, W1, alpha)
        learning_curve[i] = l5_error
        if i%10 == 0:
            print("epoch {}:\t{}".format(i, np.abs(l5_error)))
        if i%100 == 0:
            alpha *= 0.9
    return LSMR, score_vectors_dict, learning_curve

In [23]:
def fit(LSMR, score_vect_dicts,random_state=42, regressor=MLPRegressor(), classifier=MLPClassifier()):
    LSMR["score_vec"] = LSMR["Score"].apply(lambda x: score_vect_dicts[x] if x in score_vect_dicts else np.NaN)
    LSMR.dropna(inplace=True)
    
    X, Y, y = get_XYy(LSMR)
    
    regressor.random_state = random_state
    classifier.random_state = random_state
        
    regressor.fit(X, Y)
    classifier.fit(Y, y)
    return regressor, classifier

In [24]:
def predict(LSMR, score_vect_dicts, regressor, classifier):
    LSMR["score_vec"] = LSMR["Score"].apply(lambda x: score_vect_dicts[x] if x in score_vect_dicts else np.NaN)
    LSMR.dropna(inplace=True)
    
    X, Y, y = get_XYy(LSMR)
    
    preds_score_vecs = regressor.predict(X)
    pred_scores = classifier.predict(preds_score_vecs)
    
    return pred_scores, y

In [25]:
def get_separate_test_indices(df,size=100):
    tr_reviews = df[df.Language=="tr"]
    classes = list(set(tr_reviews["Score"]))
    indices = []
    for s in classes:
        indices += list(np.random.choice(tr_reviews[tr_reviews.Score==s].index, int(size/len(classes))))
    return indices

In [26]:
tr_test_indices = get_separate_test_indices(df_full)

tronly_test_raw = df_full.loc[tr_test_indices]
tronly_test = preprocess_data(tronly_test_raw)
tronly_test[tronly_test.Language=="en"].count()

Language    0
Movie_ID    0
Score       0
rev_vec     0
dtype: int64

In [27]:
tronly_test_raw.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,11,11,11
2,11,11,11
4,11,11,11
5,11,11,11
6,11,11,11
7,11,11,11
8,11,11,11
9,11,11,11
10,11,11,11


In [28]:
df = df_full.drop(tr_test_indices)
df.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,24,24,24
2,15,15,15
3,14,14,14
4,18,18,18
5,73,73,73
6,35,35,35
7,61,61,61
8,197,197,197
9,165,165,165


In [29]:
def eval_models(model, train, test, tronly, ytrain, ytest, ytronly, LL=False, predtrain=False):
    
    _ = time.time()
    model.fit(train, ytrain)
    trat = time.time()-_
    
    _ = time.time()
    predtest= model.predict(test)
    tet = time.time()-_
    s_test = distance_accuracy(ytest, predtest)
    f1_test = f1_score(ytest, predtest, average='weighted')
    
    _ = time.time()
    predtronly = model.predict(tronly)
    trt = time.time()-_
    s_tr = distance_accuracy(ytronly, predtronly)
    f1_tronly = f1_score(ytronly, predtronly, average='weighted')
    
    evals = OrderedDict()

    evals["Test"] = s_test
    evals["Tr. Only"] = s_tr
    evals["Training Time"] = trat
    evals["Testing Time"] = tet
    evals["Tr.Test Time"] = trt
    evals["F1 Test"] = f1_test
    evals["F1 Tr. only"] = f1_tronly
    if LL:
        evals["Test_LL"] = np.NAN
        evals["Tr. Only_LL"] = np.NAN
    if predtrain:
        if LL:
            evals["Train_LL"] = np.NAN
        _ = time.time()
        predicted_train = model.predict(train)
        predtra = time.time()-_
        s_train = distance_accuracy(ytrain, predicted_train)
        f1_train = f1_score(ytrain, predicted_train, average='weighted')
        evals["F1 Train"] = f1_train
        evals["Train"] = s_train
        evals["Pred.Tra. Time"] = predtra
    return evals

In [30]:
def get_total_average(scores_tables):
#     scores_tables: {i_th trial:
#                     {k_th fold:
#                         {'Model': {'Test': 0.8090301003344482,
#                                    'Train': 0.783361064891847,
#                                    'Turkish only': 0.7414285714285714}}}
    avgs = dict()
    for trial in scores_tables:
        for table in scores_tables[trial]:
            for model in scores_tables[trial][table]:
                avgs.setdefault(model, dict())
                for metric, score in scores_tables[trial][table][model].items():
                    avgs[model].setdefault(metric, list())
                    avgs[model][metric].append(score)
    for model in avgs:
        for metric in avgs[model]:
            avgs[model][metric] = np.mean(avgs[model][metric])
    return pd.DataFrame(avgs)

In [31]:
def get_trial_score(trial_scores_tables):
#  trial_scores_tables: {k_th fold:
#                             {'Model': {'Test': 0.8090301003344482,
#                                        'Train': 0.783361064891847,
#                                        'Turkish only': 0.7414285714285714}}}
    avgs = dict()
    for table in trial_scores_tables:
        for model in trial_scores_tables[table]:
            avgs.setdefault(model, dict())
            for metric, score in trial_scores_tables[table][model].items():
                avgs[model].setdefault(metric, list())
                avgs[model][metric].append(score)
    for model in avgs:
        for metric in avgs[model]:
            avgs[model][metric] = np.mean(avgs[model][metric])
    return pd.DataFrame(avgs)

In [32]:
def eval_selectivewaves_regclass(df_, tronly_test_raw, NUM_TRIALS=1, splits=10):  
    df = df_.reset_index(drop=True)
    learning_curves = dict()
    scores_tables = OrderedDict()
    tronly_test = preprocess_data(tronly_test_raw)
    for i in range(NUM_TRIALS):
        print("Trial:\t{}".format(i+1))
        scores_tables[i] = OrderedDict()
        learning_curves[i] = OrderedDict()
        k = 0
        skf = StratifiedKFold(n_splits=splits, random_state=i)
        
        for train_index, test_index in skf.split(df["Review"], df["Language"]):
            print("K:\t{}".format(k+1))
            scores_tables[i][k] = OrderedDict()
            start = time.time()
            LSMR, score_vect_dicts, training_curve = get_score_vects(
                df.loc[train_index], random_state=i, alpha=1e-5, iterations=50)
            regressor, classifier = fit(LSMR, score_vect_dicts, random_state=i)
            trat = time.time()- start

            test_data = preprocess_data(df.loc[test_index])
            _ = time.time()
            preds, true = predict(test_data, score_vect_dicts, regressor, classifier)
            tet = time.time()-_

            _ = time.time()
            preds_train, true_train = predict(preprocess_data(df.loc[train_index]),
                                              score_vect_dicts,
                                              regressor, classifier)
            predtra = time.time()-_

            _ = time.time()
            preds_tr, true_tr = predict(tronly_test, score_vect_dicts, regressor, classifier)
            trt = time.time()-_

            elapsed = time.time()-start

            s = distance_accuracy(true, preds)
            f1_test = f1_score(true, preds, average='weighted')

            s_train = distance_accuracy(true_train, preds_train)
            f1_train = f1_score(true_train, preds_train, average='weighted')

            s_tr = distance_accuracy(true_tr, preds_tr)
            f1_tronly = f1_test = f1_score(true_tr, preds_tr, average='weighted')        


            lr = LogisticRegression(random_state=i)
            mlp = MLPClassifier(random_state=i)
            rf = RandomForestClassifier(random_state=i,n_jobs=-1)
            train_mat = np.array(list(LSMR["rev_vec"]))
            test_mat = np.array(list(test_data["rev_vec"]))
            tronly_mat = np.array(list(tronly_test["rev_vec"]))

            evals = OrderedDict()
            evals["Train"] = s_train
            evals["Test"] = s
            evals["Tr. Only"] = s_tr
            evals["Training Time"] = trat
            evals["Pred.Tra. Time"] = predtra
            evals["Testing Time"] = tet
            evals["Tr.Test Time"] = trt
            evals["F1 Test"] = f1_test
            evals["F1 Train"] = f1_train
            evals["F1 Tr. only"] = f1_tronly
            scores_tables[i][k]["DeepSelect"] = evals
            scores_tables[i][k]["MLP"] = eval_models(
                mlp, train_mat, test_mat, tronly_mat, true_train, true, true_tr)
            scores_tables[i][k]["Logistic Regression"] = eval_models(
                lr, train_mat, test_mat, tronly_mat, true_train, true, true_tr)
            scores_tables[i][k]["RandomForest"] = eval_models(
                rf, train_mat, test_mat, tronly_mat, true_train, true, true_tr)

            print()
            print("K:\t{}".format(k+1))
            print(pd.DataFrame(scores_tables[i][k]))
            print("\nThis fold took:", elapsed, "seconds\n")
            learning_curves[i][k] = training_curve
            k += 1
            print("*"*10+"\n")
        print("Average scores for trial {}".format(i))
        print(get_trial_score(scores_tables[i]))
        print("-"*30)
    print("%%"*20)
    print("Average of {} trials".format(NUM_TRIALS))
    print(get_total_average(scores_tables))
    return scores_tables

In [33]:
scores_tables = eval_selectivewaves_regclass(df, tronly_test_raw)
pickle.dump(scores_tables, open("../results/batch_no_tf_tables.results", "wb"))

Trial:	1
K:	1
epoch 0:	25.956314860342978
epoch 10:	25.409497985807565
epoch 20:	24.920203411155963
epoch 30:	24.631771262147012
epoch 40:	24.42629753865604
epoch 50:	24.266966276096248


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



K:	1
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.045606  0.208007             0.177592      0.266073
F1 Tr. only       0.045606  0.063319             0.037101      0.045464
F1 Train          0.273575       NaN                  NaN           NaN
Pred.Tra. Time    1.741244       NaN                  NaN           NaN
Test              0.796894  0.807646             0.802867      0.805257
Testing Time      0.008798  0.000340             0.000133      0.103470
Tr. Only          0.569024  0.625140             0.622896      0.638608
Tr.Test Time      0.013528  0.000320             0.000103      0.105989
Train             0.820072       NaN                  NaN           NaN
Training Time    13.674850  1.327672             0.163316      0.109336

This fold took: 15.64175534248352 seconds

**********

K:	2
epoch 0:	23.029025491117427
epoch 10:	22.89758806705831
epoch 20:	22.515520809918996
epoch 30:	22.27455667965394
epoch 40:	22.0873479451234
e

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	2
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.022426  0.224415             0.194236      0.281373
F1 Tr. only       0.022426  0.056124             0.021970      0.055350
F1 Train          0.184528       NaN                  NaN           NaN
Pred.Tra. Time    1.692556       NaN                  NaN           NaN
Test              0.772043  0.803226             0.789247      0.789247
Testing Time      0.008930  0.000314             0.000153      0.103081
Tr. Only          0.531987  0.617284             0.602694      0.607183
Tr.Test Time      0.013223  0.009504             0.000100      0.106475
Train             0.810457       NaN                  NaN           NaN
Training Time    14.958243  1.117959             0.175946      0.108122

This fold took: 16.898865938186646 seconds

**********

K:	3
epoch 0:	26.620948276040583
epoch 10:	26.28512284118945
epoch 20:	25.911411611659492
epoch 30:	25.65802572650343
epoch 40:	25.41228895746455

...

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	9
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.039434  0.235938             0.225843      0.221508
F1 Tr. only       0.039434  0.051964             0.033898      0.040853
F1 Train          0.244638       NaN                  NaN           NaN
Pred.Tra. Time    1.678347       NaN                  NaN           NaN
Test              0.771739  0.803261             0.783696      0.781522
Testing Time      0.008607  0.000328             0.000149      0.104778
Tr. Only          0.569024  0.625140             0.610550      0.626263
Tr.Test Time      0.013131  0.000320             0.000106      0.105692
Train             0.817407       NaN                  NaN           NaN
Training Time    12.887814  1.312643             0.159272      0.107982

This fold took: 14.780915260314941 seconds

**********

K:	10
epoch 0:	25.817523598270167
epoch 10:	25.43769398884396
epoch 20:	25.059168437425832
epoch 30:	24.774787537323018
epoch 40:	24.514287058821

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



K:	10
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.048959  0.314952             0.335960      0.318354
F1 Tr. only       0.048959  0.047873             0.051589      0.082336
F1 Train          0.311672       NaN                  NaN           NaN
Pred.Tra. Time    1.659523       NaN                  NaN           NaN
Test              0.830918  0.830918             0.829710      0.812802
Testing Time      0.008744  0.000327             0.000121      0.103295
Tr. Only          0.643098  0.615039             0.617284      0.645342
Tr.Test Time      0.013318  0.000323             0.000099      0.105645
Train             0.834574       NaN                  NaN           NaN
Training Time    13.212426  1.183108             0.154140      0.107457

This fold took: 15.101628303527832 seconds

**********

Average scores for trial 0
                DeepSelect  Logistic Regression       MLP  RandomForest
F1 Test           0.035235             0.2516

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Using the full network for prediction
### P.S. this variation supports online (incremental) training

In [34]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [35]:
def get_test(LSMR):
    X = dict()
    y = dict()
    for _, row in LSMR.iterrows():
        score = row["Score"]
        y_ = np.zeros(10)
        y_[score-1] = 1
        y[len(y)] = y_
        X[len(X)] = row["rev_vec"]
    return np.array(list(X.values())), np.array(list(y.values()))

In [36]:
def train_selective(df_train,epochs=10, learning_rate = 0.1, random_state=42, p_every=10):
    classes = sorted(set(df_train["Score"]))
    n_classes = len(classes)
    
    LSMR_train = preprocess_data(df_train)
    np.random.seed(random_state)
    data_dict, L1, L2, L3 = get_data_dict(LSMR_train, get_L2and3=True)
    init_weights = lambda layer, i, o: {k:2*np.random.random((i, o))-1 for k in layer}
    W1 = init_weights(L1, 300, 300)  # (languge, score, movie_id)
    W2 = init_weights(L2, 300, 300)  # (languge, score):
    W3 = init_weights(L3, 300, n_classes)  # score:
    
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, n_classes]) # 1-10 => 10 classes

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, n_classes])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([n_classes]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    logits = tf.matmul(l3, w3) + b3
    pred = tf.nn.softmax(logits)

#     cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=classes, logits=logits))
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    training_curve = dict()
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            for e in range(epochs+1):
                avg_cost = 0.
                for _, row in LSMR_train.iterrows():
                    lang = row["Language"]
                    movie_id = row["Movie_ID"]
                    score = row["Score"]
                    y_ = {i:0 for i in classes}
                    y_[score] = 1
                    y_ = np.atleast_2d([y_[i] for i in classes])
                    x_ = np.atleast_2d(row["rev_vec"])
                    w1_,w2_,w3_,_, c = sess.run([w1, w2, w3, optimizer, cost],
                                             feed_dict={x: x_,
                                                        y: y_,
                                                        w1:W1[(lang, score, movie_id)],
                                                        w2:W2[(lang, score)],
                                                        w3:W3[score]})
                    W1[(lang, score, movie_id)] = w1_
                    W2[(lang, score)] = w2_
                    W3[score] = w3_

                    avg_cost += c
                training_curve[e] = avg_cost
                if e%10==0:
                    learning_rate /= 10
                if e%p_every==0:
                    print("Epoch {}: {}".format(e, avg_cost/len(LSMR_train)))
    print("W1 size",len(W1))
    print("W2 size",len(W2))
    print("W3 size",len(W3))
    return W1, W2, W3, training_curve, classes

In [37]:
def get_max_index(predicted_scores, classes):
    probs = {c:0 for c in classes}
    array = softmax(predicted_scores)
    indx = None
    max_ = float("-inf")
    for i, e in enumerate(array):
        probs[predicted_scores[i]] = e
        if e > max_:
            max_ = e
            indx = i
    return predicted_scores[indx], [probs[k] for k in sorted(classes)]

In [38]:
def predict_selective(df, W1, W2, W3, classes=list(range(1,11))):
    LSMR = preprocess_data(df)
    reset_graph()
    n_classes = len(classes)
    x = tf.placeholder(tf.float32, [None, 300])

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, n_classes])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([n_classes]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)

    
    
    
    prediction = tf.argmax(pred, 1)
    preds = np.zeros(len(LSMR))
    probs = [None] * len(LSMR)
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            j = 0
            for _, row in LSMR.iterrows():
                v = row["rev_vec"]
                predicted_scores = np.zeros(len(W1))
                for i, info in enumerate(W1):
                    language, score, movie_id = info
                    w_1 = W1[(language, score, movie_id)]
                    w_2 = W2[(language, score)]
                    w_3 = W3[score]

                    predicted_scores[i] = prediction.eval({x: np.atleast_2d(v),
                                                           w1:w_1,w2:w_2,w3:w_3})
                predicted_score, probabilities = get_max_index(predicted_scores, classes)
                preds[j] = predicted_score
                probs[j] = probabilities
                j+=1


    return preds, np.array(list(LSMR.Score)), probs

In [39]:
def eval_selectivewaves_nn(df_, tronly_test_raw, NUM_TRIALS=1, splits=10, iterations=150, alpha=0.1, p_every=25):  
    df = df_.reset_index(drop=True)
    learning_curves = OrderedDict()
    scores_tables_nn = OrderedDict()
    tronly_test = preprocess_data(tronly_test_raw)
    for i in range(NUM_TRIALS):
        print("Trial:\t{}".format(i+1))
        learning_curves[i] = OrderedDict()
        k = 0
        skf = StratifiedKFold(n_splits=splits, random_state=i)
        scores_tables_nn[i] = dict()
        for train_index, test_index in skf.split(df["Review"], df["Language"]):
            print("K: \t{}".format(k+1))
            scores_tables_nn[i][k] = OrderedDict()
            start = time.time()
            # approx 3 epochs per second
            LSMR = preprocess_data(df.loc[train_index])
            W1, W2, W3, training_curve, classes = train_selective(
                df.loc[train_index], epochs=iterations, p_every=p_every, learning_rate=alpha)
            _ = time.time()
            trat = _-start
            print("Took: {} for training".format(trat))
#             true_train = np.array(list(LSMR.Score))
            _ = time.time()
            preds_train, true_train, probs = predict_selective(df.loc[train_index], W1, W2, W3, classes=classes)
            ll_train = log_loss(true_train,probs, labels=classes)
            predtra = time.time()-_
            print("Took: {} for predicting {} training instances".format(predtra, len(train_index)))

            test_data = preprocess_data(df.loc[test_index])
            _ = time.time()
            preds, true, probs = predict_selective(df.loc[test_index], W1, W2, W3, classes=classes)
            ll_test = log_loss(true,probs, labels=classes)
            tet = time.time()-_
            print("Took: {} for predicting {} test instances".format(tet, len(test_index)))

            _ = time.time()
            preds_tr, true_tr, probs = predict_selective(tronly_test_raw, W1, W2, W3, classes=classes)
            ll_tr = log_loss(true_tr,probs, labels=classes)
            trt = time.time()-_
            print("Took: {} for predicting {} Turkish test instances".format(trt, len(tronly_test)))

            elapsed = time.time()-start

            s = distance_accuracy(true, preds)
            s_train = distance_accuracy(true_train, preds_train)
            s_tr = distance_accuracy(true_tr, preds_tr)

            f1_test = f1_score(true, preds, average='weighted')
            f1_train = f1_score(true_train, preds_train, average='weighted')
            f1_tronly = f1_score(true_tr, preds_tr, average='weighted')

            mlp = MLPClassifier(random_state=i)
            lr = LogisticRegression(random_state=i)
            rf = RandomForestClassifier(random_state=i,n_jobs=-1)
            train_mat = np.array(list(LSMR["rev_vec"]))
            test_mat = np.array(list(test_data["rev_vec"]))
            tronly_mat = np.array(list(tronly_test["rev_vec"]))

            evals = OrderedDict()
            evals["Train"] = s_train
            evals["Test"] = s
            evals["Tr. Only"] = s_tr
            evals["Training Time"] = trat
            evals["Pred.Tra. Time"] = predtra
            evals["Testing Time"] = tet
            evals["Tr.Test Time"] = trt
            evals["F1 Test"] = f1_test
            evals["F1 Train"] = f1_train
            evals["F1 Tr. only"] = f1_tronly
            evals["Train_LL"] = ll_train
            evals["Test_LL"] = ll_test
            evals["Tr. Only_LL"] = ll_tr
            scores_tables_nn[i][k]["DeepSelect"] = evals

            scores_tables_nn[i][k]["LogisticRegression"] = eval_models(
                lr, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)
            scores_tables_nn[i][k]["MLP"] = eval_models(
                mlp, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)
            scores_tables_nn[i][k]["RandomForest"] = eval_models(
                rf, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)

            print()
            print(pd.DataFrame(scores_tables_nn[i][k]))
            print("took:", elapsed, "seconds\n")
            learning_curves[i][k] = training_curve
            k += 1
            print("*"*10+"\n")
        print("Average scores for trial {}".format(i))
        print(get_trial_score(scores_tables_nn[i]))
        print("-"*30)
    print("%%"*20)
    print("Average of {} trials".format(NUM_TRIALS))
    print(get_total_average(scores_tables_nn))
    return scores_tables_nn

In [40]:
scores_tables_nn = eval_selectivewaves_nn(df, tronly_test_raw)
pickle.dump(scores_tables_nn, open("../results/incremental_tf_tables.results", "wb"))

Trial:	1
K: 	1
Epoch 0: 2.690288742985357
Epoch 25: 0.007326026052529855
Epoch 50: 0.002482170480378798
Epoch 75: 0.0015829067094770448
Epoch 100: 0.0011804284714557957
Epoch 125: 0.0009484545561848634
Epoch 150: 0.0007963821620897021
W1 size 597
W2 size 18
W3 size 10
Took: 154.26812624931335 for training
Took: 573.7708678245544 for predicting 832 training instances
Took: 64.65517807006836 for predicting 93 test instances
Took: 65.00136494636536 for predicting 99 Turkish test instances


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.152959            0.177592  0.208007      0.266073
F1 Tr. only       0.016037            0.037101  0.063319      0.045464
F1 Train          0.105947            0.336000  0.523230      0.992788
Pred.Tra. Time  573.770868            0.000599  0.001750      0.104814
Test              0.835125            0.802867  0.807646      0.805257
Test_LL          15.863970                 NaN       NaN           NaN
Testing Time     64.655178            0.000149  0.000356      0.101969
Tr. Only          0.671156            0.622896  0.625140      0.638608
Tr. Only_LL      10.375418                 NaN       NaN           NaN
Tr.Test Time     65.001365            0.000099  0.000323      0.104256
Train             0.832933            0.834976  0.868630      0.998317
Train_LL         16.151098                 NaN       NaN           NaN
Training Time   154.268126            0.149654  1.412048      0.111719
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.044803            0.194236  0.224415      0.281373
F1 Tr. only       0.022222            0.021970  0.056124      0.055350
F1 Train          0.055078            0.328084  0.537084      0.991578
Pred.Tra. Time  584.239638            0.000595  0.001612      0.106108
Test              0.809677            0.789247  0.803226      0.789247
Test_LL          16.256406                 NaN       NaN           NaN
Testing Time     64.508676            0.000136  0.000335      0.102211
Tr. Only          0.617284            0.602694  0.617284      0.607183
Tr. Only_LL       9.440916                 NaN       NaN           NaN
Tr.Test Time     66.290878            0.000102  0.000337      0.105623
Train             0.839784            0.837019  0.872115      0.997957
Train_LL         17.539208                 NaN       NaN           NaN
Training Time   154.898847            0.145544  1.373051      0.111227
took:

...


                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.070701            0.299609  0.336217      0.282909
F1 Tr. only       0.022426            0.038131  0.066138      0.060414
F1 Train          0.052277            0.327120  0.560464      0.993998
Pred.Tra. Time  568.903828            0.000632  0.001598      0.105837
Test              0.846467            0.828804  0.836957      0.815217
Test_LL          12.265711                 NaN       NaN           NaN
Testing Time     60.566497            0.000130  0.000362      0.102539
Tr. Only          0.618406            0.618406  0.621773      0.625140
Tr. Only_LL       7.165211                 NaN       NaN           NaN
Tr.Test Time     66.890208            0.000109  0.000320      0.105309
Train             0.832293            0.830252  0.878752      0.999280
Train_LL         12.402299                 NaN       NaN           NaN
Training Time   154.990288            0.150764  1.450676      0.110868
took:

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.045713            0.225843  0.235938      0.221508
F1 Tr. only       0.022426            0.033898  0.051964      0.040853
F1 Train          0.054956            0.325707  0.569586      0.995196
Pred.Tra. Time  583.724874            0.000609  0.001634      0.105671
Test              0.815217            0.783696  0.803261      0.781522
Test_LL          10.935963                 NaN       NaN           NaN
Testing Time     65.215706            0.000130  0.000352      0.105168
Tr. Only          0.618406            0.610550  0.625140      0.626263
Tr. Only_LL       7.157492                 NaN       NaN           NaN
Tr.Test Time     71.941644            0.000101  0.000324      0.105802
Train             0.839136            0.835654  0.882953      0.999160
Train_LL         12.511720                 NaN       NaN           NaN
Training Time   149.711154            0.139672  1.472392      0.111354
took:

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.034990            0.335960  0.314952      0.318354
F1 Tr. only       0.022426            0.051589  0.047873      0.082336
F1 Train          0.056317            0.325932  0.560700      0.992763
Pred.Tra. Time  544.493160            0.000587  0.001603      0.105923
Test              0.830918            0.829710  0.830918      0.812802
Test_LL          14.815081                 NaN       NaN           NaN
Testing Time     60.201206            0.000131  0.000351      0.102420
Tr. Only          0.618406            0.617284  0.615039      0.645342
Tr. Only_LL       7.166903                 NaN       NaN           NaN
Tr.Test Time     62.905035            0.000102  0.000324      0.105090
Train             0.835534            0.833133  0.880672      0.997839
Train_LL         12.129567                 NaN       NaN           NaN
Training Time   158.331020            0.147578  1.471889      0.110738
took:

# Robustness test

In [41]:
def f1_score_weighted(true, preds):
    return f1_score(true, preds, average='weighted')

In [42]:
DEFAULT_METRICES = {"MAE-accuracy":distance_accuracy, "f1_score w.avg":f1_score_weighted}

def robustness_test(df_full, test, metrics=DEFAULT_METRICES, iterations_nn=150, iterations_regclass=50):
    en_revs = df_full[df_full.Language=="en"]
    tr_revs = df_full[df_full.Language=="tr"]
    robustness = dict()  # {(num of en reviews, num of tr reviews): scores_dict}
    for en_size in range(1,11):
        for tr_size in range(1,11):
            en_train = en_revs.sample(frac=en_size/10.0)
            tr_train = tr_revs.sample(frac=tr_size/10.0)
            start = time.time()
            print("En: {}\tTr: {}".format(len(en_train),len(tr_train)))
            train = pd.concat([en_train, tr_train]).reset_index(drop=True)
            robustness_tables = dict()
            print("Using first variation (Regressor and Classifier with score vectors)")
            LSMR, score_vect_dicts, training_curve = get_score_vects(
                                            train, alpha=1e-5, iterations=iterations_regclass)
            regressor, classifier = fit(LSMR, score_vect_dicts)
            test_vecs = preprocess_data(test)
            preds, true = predict(test_vecs, score_vect_dicts, regressor, classifier)
            robustness_tables["DeepSelect (regclass)"] = dict()
            robustness_tables["DeepSelect"] = dict()

            print("Using second variation (average of outputs produced by each set of weight matrices)")
            W1, W2, W3, training_curve, classes = train_selective(train, epochs=iterations_nn, p_every=25)
            preds_nn, true_nn, probs = predict_selective(test, W1, W2, W3, classes=classes)
            ll = log_loss(true_nn,probs, labels=classes)
            
    #         f1_test_nn = f1_score(true_nn, preds_nn, average='weighted')
        
            for name, metric in metrics.items():
                
                s_regclass = metric(true, preds)
                s_nn = metric(true_nn, preds_nn)
    #         f1_test_regclass = f1_score(true, preds, average='weighted')
                robustness_tables["DeepSelect (regclass)"][name] = s_regclass
                robustness_tables["DeepSelect"][name] = s_nn
            robustness_tables["DeepSelect (regclass)"]["log loss"] = np.NAN
            robustness_tables["DeepSelect"]["log loss"] = ll

            print("Using well-known algorithms: Logistic Regression, RandomForest and MLP")
            lr = LogisticRegression()
            rf = RandomForestClassifier(n_jobs=-1)
            mlp = MLPClassifier()

            train_mat = np.array(list(LSMR["rev_vec"]))
            train_y = np.array(list(LSMR["Score"]))
            test_mat = np.array(list(test_vecs["rev_vec"]))

            for model_name, model in [("Logistic Regression",lr),
                                ("RandomForest", rf),
                                ("MLP", mlp)]:
                model.fit(train_mat, train_y)
                robustness_tables.setdefault(model_name, dict())
                robustness_tables[model_name]["log loss"] = np.NAN
                for metric_name, metric in metrics.items():
                    robustness_tables[model_name][metric_name] = metric(true, model.predict(test_mat))
            robustness[(len(en_train),len(tr_train))] = robustness_tables
            print("Took: {}".format(time.time()-start))
            print("-"*50)
    return robustness

In [43]:
robustness = robustness_test(df, tronly_test_raw)
pickle.dump(robustness, open("../results/robustness.results", "wb"))

En: 50	Tr: 42
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	16.267722696640316
epoch 10:	16.166657687382198
epoch 20:	15.91901594557589
epoch 30:	15.757097117247282
epoch 40:	15.600673612089444
epoch 50:	15.469640209091201
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 4.985744052984797
Epoch 25: 0.023468635569795126
Epoch 50: 0.010023549591386922
Epoch 75: 0.00660197866040745
Epoch 100: 0.004982926288501789
Epoch 125: 0.004028559759567468
Epoch 150: 0.0033954617488673043
W1 size 88
W2 size 15
W3 size 9


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Took: 31.7564697265625
--------------------------------------------------
En: 50	Tr: 85
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	15.823679171482095
epoch 10:	15.764763163617554
epoch 20:	15.707874642460592
epoch 30:	15.65256044064624
epoch 40:	15.593302343038062
epoch 50:	15.537421301279347
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 7.084023774737768
Epoch 25: 0.03917049839483634
Epoch 50: 0.012840683277398865
Epoch 75: 0.007969211340773823
Epoch 100: 0.005846729402185668
Epoch 125: 0.004646000576404832
Epoch 150: 0.003869292773503764
W1 size 127
W2 size 18
W3 size 10
Using well-known algorithms: Logistic Regression, RandomForest and MLP


...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Took: 247.22057175636292
--------------------------------------------------
En: 500	Tr: 425
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	24.232132378668226
epoch 10:	23.85527773362982
epoch 20:	23.330518937618347
epoch 30:	22.997198813177363
epoch 40:	22.694737047970026
epoch 50:	22.43616833897214
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 1.7484809679874602
Epoch 25: 0.004470524954066777
Epoch 50: 0.0020445942309528175
Epoch 75: 0.0013601829879409607
Epoch 100: 0.0010305221883511976
Epoch 125: 0.0008347183881057783
Epoch 150: 0.0007042807350330805
W1 size 675
W2 size 18
W3 size 10
Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 272.48110246658325
--------------------------------------------------


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# grouping labels
### label = 3   if review_score > 7
### label = 2   if 7 >= review_score >= 4
### label = 1  if review_score < 4

In [51]:
def group_labels(x):
    if x>7:
        return 3
    elif x>=4:
        return 2
    return 1

In [52]:
df_full_tri = deepcopy(df_full)
df_full_tri["Score"] = df_full_tri.Score.apply(group_labels)
df_full_tri.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,64,64,64
2,220,220,220
3,716,716,716


In [53]:
tr_tri_test_indices = get_separate_test_indices(df_full_tri)

tronly_test_raw_tri = df_full_tri.loc[tr_tri_test_indices]
tronly_test_tri = preprocess_data(tronly_test_raw_tri)
tronly_test_tri[tronly_test_tri.Language=="en"].count()

Language    0
Movie_ID    0
Score       0
rev_vec     0
dtype: int64

In [54]:
df_tri = df_full_tri.drop(tr_tri_test_indices)
df_tri.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,46,46,46
2,193,193,193
3,686,686,686


In [55]:
scores_tables_tri = eval_selectivewaves_regclass(df_tri, tronly_test_raw_tri)
pickle.dump(scores_tables_tri, open("../results/batch_no_tf_tables_tri.results", "wb"))

Trial:	1
K:	1
epoch 0:	20.718364773349318
epoch 10:	20.38550746253965
epoch 20:	20.338702648861954
epoch 30:	20.215794532591975
epoch 40:	20.085571056721765
epoch 50:	19.951271865752815


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	1
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.650281             0.632019      0.648394
F1 Tr. only       0.166667  0.208791             0.166667      0.210446
F1 Train          0.631551       NaN                  NaN           NaN
Pred.Tra. Time    1.746940       NaN                  NaN           NaN
Test              0.903226  0.903226             0.903226      0.874552
Testing Time      0.009262  0.000287             0.000151      0.103292
Tr. Only          0.666667  0.686869             0.666667      0.680135
Tr.Test Time      0.015023  0.000285             0.000130      0.105254
Train             0.896635       NaN                  NaN           NaN
Training Time    13.393996  0.496128             0.057576      0.111610

This fold took: 15.36376142501831 seconds

**********

K:	2
epoch 0:	20.372812676665088
epoch 10:	19.76452811105361
epoch 20:	19.555187235853193
epoch 30:	19.26816104670939
epoch 40:	19.00539586203371


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	2
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.582091             0.533333      0.603629
F1 Tr. only       0.166667  0.187547             0.166667      0.198209
F1 Train          0.642857       NaN                  NaN           NaN
Pred.Tra. Time    1.664287       NaN                  NaN           NaN
Test              0.863799  0.874552             0.863799      0.867384
Testing Time      0.008765  0.000339             0.000148      0.103173
Tr. Only          0.666667  0.670034             0.666667      0.673401
Tr.Test Time      0.013004  0.000291             0.000123      0.108889
Train             0.901042       NaN                  NaN           NaN
Training Time    10.888238  0.704684             0.055434      0.110970

This fold took: 12.797622442245483 seconds

**********

K:	3
epoch 0:	21.100234276326542
epoch 10:	20.912934057089487
epoch 20:	20.57607062887099
epoch 30:	20.252583327905246
epoch 40:	19.9645017827957

...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	8
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.703118             0.717129      0.703806
F1 Tr. only       0.166667  0.166667             0.166667      0.182927
F1 Train          0.622329       NaN                  NaN           NaN
Pred.Tra. Time    2.146744       NaN                  NaN           NaN
Test              0.927536  0.909420             0.927536      0.905797
Testing Time      0.008819  0.000427             0.000271      0.101978
Tr. Only          0.666667  0.666667             0.666667      0.680135
Tr.Test Time      0.019070  0.000433             0.000104      0.102628
Train             0.893958       NaN                  NaN           NaN
Training Time    11.942627  1.641979             0.095596      0.109195

This fold took: 14.357232809066772 seconds

**********

K:	9
epoch 0:	20.62122102142833
epoch 10:	20.65986344006847
epoch 20:	20.496337207415863
epoch 30:	20.351596573728255
epoch 40:	20.21391590200612

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	9
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.662960             0.608420      0.592142
F1 Tr. only       0.166667  0.208791             0.166667      0.212045
F1 Train          0.633577       NaN                  NaN           NaN
Pred.Tra. Time    1.789888       NaN                  NaN           NaN
Test              0.880435  0.884058             0.876812      0.847826
Testing Time      0.008938  0.000332             0.000217      0.103046
Tr. Only          0.666667  0.686869             0.666667      0.693603
Tr.Test Time      0.013258  0.000300             0.000124      0.105238
Train             0.899160       NaN                  NaN           NaN
Training Time    12.055574  0.793447             0.055322      0.111056

This fold took: 14.057385444641113 seconds

**********

K:	10
epoch 0:	20.512011650724705
epoch 10:	20.071439447274173
epoch 20:	19.87600472599945
epoch 30:	19.71451899041192
epoch 40:	19.5851294904519

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [56]:
scores_tables_nn_tri = eval_selectivewaves_nn(df_tri, tronly_test_raw_tri)
pickle.dump(scores_tables_nn_tri, open("../results/incremental_tf_tables_tri.results", "wb"))

Trial:	1
K: 	1
Epoch 0: 0.0991042266822565
Epoch 25: 0.00033590976964267187
Epoch 50: 0.00017458095181892392
Epoch 75: 0.00011954870702922927
Epoch 100: 9.1475841480094e-05
Epoch 125: 7.43621997850804e-05
Epoch 150: 6.280544335637636e-05
W1 size 426
W2 size 6
W3 size 3
Took: 151.9141550064087 for training
Took: 363.7457580566406 for predicting 832 training instances
Took: 42.369715452194214 for predicting 93 test instances
Took: 46.19421982765198 for predicting 99 Turkish test instances


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.083192            0.632019  0.650281      0.648394
F1 Tr. only       0.166667            0.166667  0.208791      0.210446
F1 Train          0.070490            0.637177  0.697684      0.995172
Pred.Tra. Time  363.745758            0.000569  0.001353      0.106071
Test              0.741935            0.903226  0.903226      0.874552
Test_LL          22.181999                 NaN       NaN           NaN
Testing Time     42.369715            0.000170  0.000335      0.104240
Tr. Only          0.777778            0.666667  0.686869      0.680135
Tr. Only_LL      10.455074                 NaN       NaN           NaN
Tr.Test Time     46.194220            0.000094  0.000306      0.105189
Train             0.735176            0.897436  0.906250      0.998397
Train_LL         22.225025                 NaN       NaN           NaN
Training Time   151.914155            0.054760  0.472875      0.111123
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.105873            0.533333  0.582091      0.603629
F1 Tr. only       0.166667            0.166667  0.187547      0.198209
F1 Train          0.068588            0.648493  0.718635      0.996384
Pred.Tra. Time  398.527356            0.000547  0.001337      0.105774
Test              0.752688            0.863799  0.874552      0.867384
Test_LL          20.010921                 NaN       NaN           NaN
Testing Time     47.710488            0.000132  0.000334      0.102402
Tr. Only          0.777778            0.666667  0.670034      0.673401
Tr. Only_LL      10.453219                 NaN       NaN           NaN
Tr.Test Time     42.412308            0.000094  0.000303      0.105440
Train             0.734375            0.901843  0.913061      0.998397
Train_LL         22.435100                 NaN       NaN           NaN
Training Time   144.489873            0.051529  0.573318      0.111192
took:

...


                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.051530            0.717129  0.703118      0.703806
F1 Tr. only       0.166667            0.166667  0.166667      0.182927
F1 Train          0.074475            0.627940  0.773475      0.995178
Pred.Tra. Time  390.336936            0.000520  0.001329      0.105955
Test              0.724638            0.927536  0.909420      0.905797
Test_LL          23.592207                 NaN       NaN           NaN
Testing Time     45.368505            0.000161  0.000328      0.102179
Tr. Only          0.777778            0.666667  0.666667      0.680135
Tr. Only_LL      10.289200                 NaN       NaN           NaN
Tr.Test Time     40.060471            0.000094  0.000301      0.103654
Train             0.737495            0.894758  0.923169      0.998399
Train_LL         21.610089                 NaN       NaN           NaN
Training Time   141.081370            0.056207  0.730671      0.111103
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.057639            0.608420  0.662960      0.592142
F1 Tr. only       0.166667            0.166667  0.208791      0.212045
F1 Train          0.073709            0.639199  0.778288      0.996387
Pred.Tra. Time  382.251837            0.000550  0.001324      0.105908
Test              0.728261            0.876812  0.884058      0.847826
Test_LL          21.554698                 NaN       NaN           NaN
Testing Time     40.532547            0.000146  0.000285      0.102673
Tr. Only          0.777778            0.666667  0.686869      0.693603
Tr. Only_LL      10.322407                 NaN       NaN           NaN
Tr.Test Time     51.332402            0.000096  0.000308      0.104917
Train             0.737095            0.899960  0.927171      0.998800
Train_LL         21.948745                 NaN       NaN           NaN
Training Time   140.488700            0.050765  0.729338      0.111878
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.051530            0.702108  0.753901      0.744423
F1 Tr. only       0.166667            0.166667  0.171875      0.203857
F1 Train          0.074475            0.629545  0.769983      0.990358
Pred.Tra. Time  394.404755            0.000534  0.001345      0.105475
Test              0.724638            0.920290  0.923913      0.909420
Test_LL          23.642418                 NaN       NaN           NaN
Testing Time     43.688625            0.000136  0.000282      0.101942
Tr. Only          0.777778            0.666667  0.680135      0.656566
Tr. Only_LL      10.439230                 NaN       NaN           NaN
Tr.Test Time     49.369885            0.000095  0.000287      0.104699
Train             0.737495            0.895558  0.923169      0.996399
Train_LL         21.972381                 NaN       NaN           NaN
Training Time   141.797365            0.055685  0.747958      0.111627
took:

In [57]:
robustness_tri = robustness_test(df_tri, tronly_test_raw_tri)
pickle.dump(robustness_tri, open("../results/robustness_tri.results", "wb"))

En: 50	Tr: 42
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	14.964425073833835
epoch 10:	14.863251305838217
epoch 20:	14.671541291477538
epoch 30:	14.614800065274787
epoch 40:	14.452866451696048
epoch 50:	14.293558946326295
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 0.7156531736173027
Epoch 25: 0.003076515007679943
Epoch 50: 0.0015267655560438945
Epoch 75: 0.0010261519677678289
Epoch 100: 0.0007762732542945093
Epoch 125: 0.0006258429243671864
Epoch 150: 0.0005251219510887371
W1 size 82
W2 size 6
W3 size 3


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Took: 29.403439044952393
--------------------------------------------------
En: 50	Tr: 85
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	16.076561720338827
epoch 10:	15.983979964190306
epoch 20:	15.858078810358194
epoch 30:	15.721506107987842
epoch 40:	15.600251133478585
epoch 50:	15.480006247684997
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 2.503276165758251
Epoch 25: 0.003296139311705217
Epoch 50: 0.0015734759895093685
Epoch 75: 0.0010588947022489071
Epoch 100: 0.0008049237510275757
Epoch 125: 0.0006521573283707641
Epoch 150: 0.0005496701311603535
W1 size 113
W2 size 6
W3 size 3
Using well-known algorithms: Logistic Regression, RandomForest and MLP


...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 233.84322881698608
--------------------------------------------------
En: 500	Tr: 425
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	20.415601593774113
epoch 10:	20.176899096379984
epoch 20:	19.91497524803913
epoch 30:	19.73435325182928
epoch 40:	19.588685070642267
epoch 50:	19.466115146914195
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 0.33409296077505946
Epoch 25: 0.0005557524643278446
Epoch 50: 0.0002814696849764834
Epoch 75: 0.00019031503163314422
Epoch 100: 0.00014437298408533688
Epoch 125: 0.00011658674632050824
Epoch 150: 9.791964096862287e-05
W1 size 485
W2 size 6
W3 size 3
Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 231.13006258010864
--------------------------------------------------


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
# HOW TO CHEAT LIKE A PRO
# """
# def test_selective(df_test, W1, W2, W3):
#     reset_graph()
#     x = tf.placeholder(tf.float32, [None, 300])
#     y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

#     w1 = tf.placeholder(tf.float32, [300, 300])
#     w2 = tf.placeholder(tf.float32, [300, 300])
#     w3 = tf.placeholder(tf.float32, [300, 10])

#     b1 = tf.Variable(tf.zeros([300]))
#     b2 = tf.Variable(tf.zeros([300]))
#     b3 = tf.Variable(tf.zeros([10]))

#     l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
#     l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
#     pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)
    
#     correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#     instance_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
    
#         # Testing the model
#         LSMR_test = preprocess_data(df_test)
#         X_test, y_test = get_test(LSMR_test)
#         accuracy = 0.
#         for i in range(len(X_test)):
#             best_instance_accuracy = float("-inf")
#             for language, score, movie_id in W1:
#                 w_1 = W1[(language, score, movie_id)]
#                 w_2 = W2[(language, score)]
#                 w_3 = W3[score]
#                 a = instance_accuracy.eval({x: np.atleast_2d(X_test[i]), y: np.atleast_2d(y_test[i]),
#                                    w1:w_1,
#                                    w2:w_2,
#                                    w3:w_3})
#                 if a > best_instance_accuracy:
#                     best_instance_accuracy = a
#             accuracy += best_instance_accuracy

#     return accuracy/len(X_test)
# """

# 3-layer NN > needs at least 3 days for training

In [ ]:
# gpu is a must
def train_deep(df_train, epochs=100, learning_rate=0.1, random_state=42):
    LSMR_train = preprocess_data(df_train)
    np.random.seed(random_state)
    data_dict, L1, L2, L3 = get_data_dict(LSMR_train, get_L2and3=True)
    init_weights = lambda layer, i, o: {k:2*np.random.random((i, o))-1 for k in layer}
    W1 = init_weights(L1, 300, 300)  # (languge, score, movie_id)
    W2 = init_weights(L2, 300, 300)  # (languge, score):
    W3 = init_weights(L3, 300, 10)  # score:
    
    
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

    w1 = tf.Variable(tf.zeros([300, 300]))
    w2 = tf.Variable(tf.zeros([300, 300]))
    w3 = tf.Variable(tf.zeros([300, 10]))

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([10]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)


    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    training_curve = dict()
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            for e in range(epochs+1):
                start = time.time()
                avg_cost = 0.
                for _, row in LSMR_train.iterrows():
                    score = row["Score"]
                    y_ = np.zeros(10)
                    y_[score-1] = 1
                    y_ = np.atleast_2d(y_)
                    x_ = np.atleast_2d(row["rev_vec"])
                    w_1, w_2, w_3 , _, c = sess.run([w1, w2, w3, optimizer, cost], feed_dict={x: x_,y: y_})               
                    avg_cost += c
                avg_cost /= len(LSMR_train)
                training_curve[e] = (avg_cost, time.time()-start)
                if e%10==0:
                    print("Epoch {}: {}".format(e, avg_cost))

    return w_1, w_2, w_3, training_curve

In [ ]:
def test_deep(df_test, w_1, w_2, w_3):
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, 10])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([10]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())

            # Testing the model
            LSMR_test = preprocess_data(df_test)
            X_test, y_test = get_test(LSMR_test)
            return accuracy.eval({x: X_test,
                                  y: y_test,
                                  w1:w_1,w2:w_2,
                                  w3:w_3})

In [ ]:
NUM_TRIALS = 1
scores_incremental = dict()
learning_curves = dict()
for i in range(NUM_TRIALS):
    scores_incremental[i] = dict()
    learning_curves[i] = dict()
    print("Trial:\t{}".format(i+1))
    k = 0
    skf = StratifiedKFold(n_splits=10, random_state=i)
    for train_index, test_index in skf.split(df["Review"], df["Language"]):
        start = time.time()
        w1, w2, w3, learning_curve = train_deep(df.loc[train_index], random_state=i, epochs=10000)
        s = test_deep(df.loc[test_index], w1, w2, w3)
        k += 1
        print("K:\t{}\nScore:\t{}".format(k, s))
        print("took:", time.time()-start)
        scores_incremental[i][k] = s
        learning_curves[i][k] = learning_curve
    print("*"*10)
    try:
        print("Trial {} avg score:\t {}".format(i+1, np.mean(list(scores_incremental[i].values()))))
    except:
        continue
    print("-"*30)